# Amazon Bedrock AgentCore Policy - Getting Started Demo

## Overview

Welcome to the Amazon Bedrock AgentCore Policy hands-on demo! This notebook will guide you through the complete workflow of setting up and testing policy-based security controls for AI agent interactions.

### What is AgentCore Policy?

Amazon Bedrock AgentCore Policy enables developers to define and enforce security controls for AI agent interactions with tools by creating a protective boundary ("safety box") around agent operations. AI agents can dynamically adapt to solve complex problems, but this flexibility introduces security challenges:

- **Data Leakage**: Agents may inadvertently expose private information
- **Business Rule Violations**: Agents might misinterpret or bypass business rules
- **Authority Overreach**: Agents could act outside their intended scope

Policy intercepts inbound agent traffic through AgentCore Gateways and evaluates each request against defined policies before allowing tool access.

### Key Benefits

✅ **Declarative Security**: Define policies using Cedar language, not code  
✅ **Runtime Enforcement**: Policies are evaluated in real-time  
✅ **Fine-Grained Control**: From coarse restrictions to detailed transaction limits  
✅ **Separation of Concerns**: Security logic lives outside agent code  
✅ **Enterprise Scale**: Deploy autonomous agents safely in production  

---

## Demo Architecture

```
┌─────────────┐
│   AI Agent  │
└──────┬──────┘
       │
       │ Tool Call Request
       ▼
┌─────────────────────┐
│  AgentCore Gateway  │
│  + OAuth Auth       │
└──────┬──────────────┘
       │
       │ Policy Check
       ▼
┌─────────────────────┐
│   Policy Engine     │
│   (Cedar Policies)  │
└──────┬──────────────┘
       │
       │ ALLOW / DENY
       ▼
┌─────────────────────┐
│   Lambda Target     │
│   (RefundTool)      │
└─────────────────────┘
```

---

## What You'll Learn

In this demo, you will:

1. **Setup Infrastructure**: Create a Gateway with Lambda targets
2. **Create Policy Engine**: Initialize a policy engine for your gateway
3. **Define Policies**: Write Cedar policies to control access
4. **Test Enforcement**: Verify policies work with real agent requests
5. **Understand Results**: Interpret ALLOW and DENY scenarios

---

## Prerequisites

Before starting, ensure you have:

- ✅ AWS CLI configured with appropriate credentials
- ✅ Python 3.10+ with boto3 installed
- ✅ `bedrock_agentcore_starter_toolkit` package installed
- ✅ Access to AWS Lambda (for target functions)
- ✅ IAM role with trust policy for `preprod.genesis-service.aws.internal`
- ✅ Working in **us-east-1 (N.Virginia)** region

---

## Demo Scenario: Refund Processing

We'll implement a **refund processing system** with policy controls:

- **Tool**: `RefundTool` - Processes customer refunds
- **Parameters**: `amount` (integer), `orderId` (string)
- **Policy Rule**: Only allow refunds under $1000
- **Test Cases**: 
  - ✅ $200 refund (should be ALLOWED)
  - ❌ $2000 refund (should be DENIED)

Let's get started! 🚀

---

# Step 0: Environment Setup

First, let's verify our environment and import necessary libraries.

In [ ]:
# Import required libraries
import json
import boto3
import sys
import os
from pathlib import Path

# Add the scripts directory to Python path
scripts_dir = Path.cwd() / 'scripts'
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

# Verify region
session = boto3.Session()
region = session.region_name or 'us-east-1'

# Verify AWS credentials
try:
    sts = session.client('sts')
    identity = sts.get_caller_identity()
    print("✅ AWS Credentials Verified")
    print(f"   Account: {identity['Account']}")
    print(f"   User/Role: {identity['Arn']}")
except Exception as e:
    print(f"❌ AWS Credentials Error: {e}")
    print("   Please configure AWS CLI with: aws configure")


print(f"\n📍 Region: {region}")
if region != 'us-east-1':
    print("⚠️  Warning: This demo is designed for us-east-1")

# Check for required packages
try:
    from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
    print("\n✅ bedrock_agentcore_starter_toolkit installed")
except ImportError:
    print("\n❌ bedrock_agentcore_starter_toolkit not found")
    print("   Install with: pip install bedrock-agentcore-starter-toolkit")

---

# Step 1: Create Lambda Target Function

Before setting up the gateway, we need a Lambda function that will serve as our tool target.

## What is a Lambda Target?

A Lambda target is a backend function that the AI agent can invoke through the gateway. In our case, it's a simple refund processing function.

## Lambda Function Code

The Lambda function should:
- Accept `amount` and `orderId` parameters
- Return a success response with refund details
- Handle both direct invocation and gateway formats

### Option 1: Create via AWS Console

1. Go to AWS Lambda Console in **us-east-1**
2. Click "Create function"
3. Name: `RefundTool`
4. Runtime: Node.js 24.x
5. Use the code below

### Option 2: Create via CLI (run the cell below)

In [ ]:
# Lambda function code for RefundTool
lambda_code = '''
console.log('Loading function');

export const handler = async (event, context) => {
    console.log('event =', JSON.stringify(event));
    console.log('context =', JSON.stringify(context));
    
    var response = undefined;
    
    if (event.body !== undefined) {
        // API Gateway format
        console.log('event.body =', event.body);
        const body = JSON.parse(event.body);
        response = {
            "status": "SUCCESS",
            "message": `Refund processed successfully: $${body.amount} for order ${body.orderId}`,
            "amount": body.amount,
            "orderId": body.orderId
        };
    } else {
        // Direct invocation from Gateway
        response = {
            "status": "SUCCESS",
            "message": `Refund processed successfully: $${event.amount} for order ${event.orderId}`,
            "amount": event.amount,
            "orderId": event.orderId
        };
        return response;
    }
    
    console.log('response =', JSON.stringify(response));
    return {"statusCode": 200, "body": JSON.stringify(response)};
};
'''

# Save Lambda code to file
lambda_file = scripts_dir / 'refund_tool.mjs'
lambda_file.parent.mkdir(exist_ok=True)
with open(lambda_file, 'w') as f:
    f.write(lambda_code)

print("✅ Lambda function code saved to:", lambda_file)
print("\n📋 Next steps:")
print("   1. Create Lambda function named 'RefundTool' in AWS Console")
print("   2. Use Node.js 24.x runtime")
print("   3. Copy the code above into the function")
print("   4. Note the Lambda ARN for the next step")
print("\n💡 Or deploy using AWS CLI:")
print(f"   cd {scripts_dir}")
print("   zip refund_tool.zip refund_tool.mjs")
print("   aws lambda create-function --function-name RefundTool \\")
print("       --runtime nodejs24.x --role <YOUR_LAMBDA_ROLE_ARN> \\")
print("       --handler index.handler --zip-file fileb://refund_tool.zip")

### Verify Lambda Function

Once created, let's verify the Lambda function exists:

In [ ]:
# Check if Lambda function exists
lambda_client = boto3.client('lambda', region_name='us-east-1')

try:
    response = lambda_client.get_function(FunctionName='RefundTool')
    lambda_arn = response['Configuration']['FunctionArn']
    print("✅ Lambda function found!")
    print(f"   Function Name: RefundTool")
    print(f"   ARN: {lambda_arn}")
    print(f"   Runtime: {response['Configuration']['Runtime']}")
    print(f"   Handler: {response['Configuration']['Handler']}")
    
    # Save Lambda ARN for later use
    config_file = scripts_dir / 'lambda_config.json'
    with open(config_file, 'w') as f:
        json.dump({'lambda_arn': lambda_arn}, f, indent=2)
    print(f"\n💾 Lambda ARN saved to: {config_file}")
    
except lambda_client.exceptions.ResourceNotFoundException:
    print("❌ Lambda function 'RefundTool' not found")
    print("   Please create it first using the instructions above")
    lambda_arn = None
except Exception as e:
    print(f"❌ Error checking Lambda: {e}")
    lambda_arn = None

---

# Step 2: Setup AgentCore Gateway

Now we'll create an AgentCore Gateway with OAuth authentication and attach our Lambda function as a target.

## What Gets Created?

1. **OAuth Authorization Server**: Cognito-based OAuth for authentication
2. **AgentCore Gateway**: Main gateway with MCP protocol support
3. **Lambda Target**: Your RefundTool attached with schema definition
4. **Configuration File**: All connection details saved for later use

## Gateway Configuration

The gateway will be configured with:
- **Protocol**: MCP (Model Context Protocol)
- **Authentication**: OAuth 2.0 via Cognito
- **Target**: RefundTool Lambda function
- **Tool Schema**: Defines `amount` and `orderId` parameters

In [ ]:
# Run the gateway setup script
print("🚀 Setting up AgentCore Gateway...\n")
print("This will:")
print("  1. Create OAuth authorization server (Cognito)")
print("  2. Create AgentCore Gateway")
print("  3. Attach RefundTool Lambda as target")
print("  4. Save configuration to gateway_config.json")
print("\n" + "="*60)

# Import and run the setup script
import setup_gateway

# The script will prompt for:
# - Role ARN (IAM role with trust policy)
# - Lambda ARN (from previous step)

print("\n💡 You will be prompted for:")
print("   1. IAM Role ARN (with trust policy for AgentCore added to the IAM Role)")
if lambda_arn:
    print(f"   2. Lambda ARN: {lambda_arn}")
else:
    print("   2. Lambda ARN (from your RefundTool function)")

In [ ]:
# Run the gateway setup
%run scripts/setup_gateway.py

### Verify Gateway Configuration

Let's load and verify the gateway configuration that was just created:

In [ ]:
# Load gateway configuration
gateway_config_file = scripts_dir / 'gateway_config.json'

if gateway_config_file.exists():
    with open(gateway_config_file, 'r') as f:
        gateway_config = json.load(f)
    
    print("✅ Gateway Configuration Loaded\n")
    print("="*60)
    print(f"Gateway ID:  {gateway_config['gateway_id']}")
    print(f"Gateway ARN: {gateway_config['gateway_arn']}")
    print(f"Gateway URL: {gateway_config['gateway_url']}")
    print(f"Region:      {gateway_config['region']}")
    print("\nOAuth Configuration:")
    print(f"  Client ID:  {gateway_config['client_info']['client_id']}")
    print(f"  Token URL:  {gateway_config['client_info']['token_endpoint']}")
    print("="*60)
    
    # Store for later use
    GATEWAY_ARN = gateway_config['gateway_arn']
    GATEWAY_ID = gateway_config['gateway_id']
    GATEWAY_URL = gateway_config['gateway_url']
    
else:
    print("❌ Gateway configuration not found")
    print("   Please run the gateway setup cell above first")

---

# Step 3: Create Policy Engine and Policies

Now we'll create a Policy Engine with Cedar policies to control access to our RefundTool.

## What is a Policy Engine?

A Policy Engine evaluates requests against Cedar policies in real-time. It operates in two modes:
- **LOG_ONLY**: Evaluates but doesn't block (for testing)
- **ENFORCE**: Actively blocks non-compliant requests (for production)

## Our Policy Rule

We'll create a policy that only allows refunds under $1000:

```cedar
permit(
  principal,
  action == AgentCore::Action::"RefundToolTarget___refund",
  resource == AgentCore::Gateway::"<gateway-arn>"
) when {
  context.input.amount <= 1000
};
```

This means:
- ✅ Refunds of $1000 or less: **ALLOWED**
- ❌ Refunds over $1000: **DENIED**

### Create Policy Engine

First, we'll create a Policy Engine to hold our Cedar policies:

In [ ]:
# Import PolicyGenerator
import sys
sys.path.insert(0, str(scripts_dir))
from policy_generator import PolicyGenerator

# Initialize PolicyGenerator
print("🔧 Initializing Policy Generator...")
generator = PolicyGenerator()
print("✅ Policy Generator initialized\n")

# Check if policy engine already exists
policy_engine_id = gateway_config.get('policy_engine_id')

if policy_engine_id:
    print(f"✅ Using existing Policy Engine: {policy_engine_id}")
else:
    # Create new policy engine
    print("📝 Creating new Policy Engine...")
    import time
    engine_name = f"PolicyEngine_{int(time.time())}"
    policy_engine_id = generator.create_policy_engine(engine_name)
    
    if policy_engine_id:
        print(f"✅ Policy Engine created: {policy_engine_id}")
        
        # Wait for it to become active
        print("⏳ Waiting for Policy Engine to become ACTIVE...")
        if generator.wait_for_policy_engine_active(policy_engine_id):
            print("✅ Policy Engine is ACTIVE")
        
        # Save to config
        gateway_config['policy_engine_id'] = policy_engine_id
        with open(gateway_config_file, 'w') as f:
            json.dump(gateway_config, f, indent=2)
    else:
        print("❌ Failed to create Policy Engine")

### Create Cedar Policy

Now we'll create a Cedar policy that allows refunds under $1000:

In [ ]:
# Create Cedar policy
print("\n📝 Creating Cedar Policy...")
print(f"   Policy Engine ID: {policy_engine_id}")
print(f"   Gateway ARN: {GATEWAY_ARN}")

# Define the Cedar policy statement
cedar_statement = (
    f'permit(principal, '
    f'action == AgentCore::Action::"RefundToolTarget___refund", '
    f'resource == AgentCore::Gateway::"{GATEWAY_ARN}") '
    f'when {{ context.input.amount <= 1000 }};'
)

print("\n📋 Policy Statement:")
print(cedar_statement)

# Create the policy
try:
    policy_response = generator.client.create_policy(
        policyEngineId=policy_engine_id,
        name='refundspolicy1',
        description='Allow refunds under $1000',
        definition={
            'cedar': {
                'statement': cedar_statement
            }
        }
    )
    
    policy_id = policy_response['policyId']
    policy_engine_arn = policy_response.get('policyEngineArn')
    
    print("\n✅ Policy Created Successfully!")
    print(f"   Policy ID: {policy_id}")
    print(f"   Status: {policy_response['status']}")
    
    # Save to config
    gateway_config['policy_id'] = policy_id
    gateway_config['policy_engine_arn'] = policy_engine_arn
    with open(gateway_config_file, 'w') as f:
        json.dump(gateway_config, f, indent=2)
    
    POLICY_ENGINE_ARN = policy_engine_arn
    
except Exception as e:
    print(f"\n❌ Error creating policy: {e}")

### Verify Policy Creation

Let's verify the policy engine and policy were created:

In [ ]:
# Reload gateway configuration with policy details
with open(gateway_config_file, 'r') as f:
    gateway_config = json.load(f)

if 'policy_engine_id' in gateway_config:
    print("✅ Policy Engine and Policy Created\n")
    print("="*60)
    print(f"Policy Engine ID:  {gateway_config.get('policy_engine_id')}")
    if 'policy_engine_arn' in gateway_config:
        print(f"Policy Engine ARN: {gateway_config['policy_engine_arn']}")
    if 'policy_id' in gateway_config:
        print(f"Policy ID:         {gateway_config['policy_id']}")
    print("="*60)
    
    print("\n📋 Policy Details:")
    print("  Name: refundspolicy1")
    print("  Description: Allow refunds under $1000")
    print("  Type: Cedar Policy")
    print("  Status: ACTIVE")
    
    print("\n📝 Policy Rule:")
    print("  Refunds <= $1000: ALLOWED ✅")
    print("  Refunds > $1000:  DENIED ❌")
    
    POLICY_ENGINE_ARN = gateway_config.get('policy_engine_arn')
    POLICY_ENGINE_ID = gateway_config.get('policy_engine_id')
else:
    print("❌ Policy Engine not found")
    print("   Please run the policy creation cell above")

---

# Step 4: Test Policy Enforcement with AI Agent

Now for the exciting part - let's test our policy with a real AI agent!

## Test Scenarios

We'll test two scenarios:

### Test 1: ALLOWED Scenario ✅
- **Request**: Process a $200 refund
- **Expected**: Policy allows, Lambda executes, refund processed
- **Reason**: $200 <= $1000 (within policy limit)

### Test 2: DENIED Scenario ❌
- **Request**: Process a $2000 refund
- **Expected**: Policy blocks, Lambda never executes
- **Reason**: $2000 > $1000 (exceeds policy limit)

## How It Works

```
Agent → Gateway → Policy Engine → Decision
                       ↓
                  Check: amount <= 1000?
                       ↓
              YES: Allow → Lambda
              NO:  Deny → Block
```

### Test 1: ALLOWED - $200 Refund

This should be **ALLOWED** by the policy:

In [ ]:
# Test 1: ALLOWED scenario - $200 refund
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client
import requests

print("🧪 Test 1: ALLOWED Scenario")
print("="*60)
print("Request: Process a $200 refund for order test-allow")
print("Expected: ALLOWED (200 <= 1000)")
print("="*60)
print()

# Get OAuth token
print("🔑 Getting access token...")
token_response = requests.post(
    gateway_config['client_info']['token_endpoint'],
    data=f"grant_type=client_credentials&client_id={gateway_config['client_info']['client_id']}&client_secret={gateway_config['client_info']['client_secret']}",
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
)
access_token = token_response.json()['access_token']
print("✅ Access token obtained\n")

# Setup MCP transport
def create_transport(url, token):
    return streamablehttp_client(url, headers={"Authorization": f"Bearer {token}"})

# Setup Bedrock model
model = BedrockModel(
    inference_profile_id="amazon.nova-lite-v1:0",
    streaming=True
)

# Create MCP client and agent
mcp_client = MCPClient(lambda: create_transport(GATEWAY_URL, access_token))

with mcp_client:
    # List tools
    tools = mcp_client.list_tools_sync()
    print(f"📋 Available tools: {[tool.tool_name for tool in tools]}\n")
    
    # Create agent
    agent = Agent(model=model, tools=tools)
    
    # Run test
    print("💬 Prompt: Process a refund of $200 for order test-allow")
    print("🤔 Thinking...\n")
    
    response = agent("Process a refund of $200 for order test-allow")
    
    print("\n" + "="*60)
    print("✅ Test 1 Complete - Request was ALLOWED")
    print("="*60)

### Test 2: DENIED - $2000 Refund

This should be **DENIED** by the policy:

In [ ]:
# Test 1: DENIED scenario - $2000 refund
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client
import requests

print("🧪 Test 2: DENIED Scenario")
print("="*60)
print("Request: Process a $2000 refund for order test-deny")
print("Expected: DENIED (2000 > 1000)")
print("="*60)
print()

# Get OAuth token
print("🔑 Getting access token...")
token_response = requests.post(
    gateway_config['client_info']['token_endpoint'],
    data=f"grant_type=client_credentials&client_id={gateway_config['client_info']['client_id']}&client_secret={gateway_config['client_info']['client_secret']}",
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
)
access_token = token_response.json()['access_token']
print("✅ Access token obtained\n")

# Setup MCP transport
def create_transport(url, token):
    return streamablehttp_client(url, headers={"Authorization": f"Bearer {token}"})

# Setup Bedrock model
model = BedrockModel(
    inference_profile_id="amazon.nova-lite-v1:0",
    streaming=True
)

# Create MCP client and agent
mcp_client = MCPClient(lambda: create_transport(GATEWAY_URL, access_token))

with mcp_client:
    # List tools
    tools = mcp_client.list_tools_sync()
    print(f"📋 Available tools: {[tool.tool_name for tool in tools]}\n")
    
    # Create agent
    agent = Agent(model=model, tools=tools)
    
    # Run test
    print("💬 Prompt: Process a refund of $2000 for order test-deny")
    print("🤔 Thinking...\n")
    
    response = agent("Process a refund of $2000 for order test-deny")
    
    print("\n" + "="*60)
    print("✅ Test 1 Complete - Request was DENIED")
    print("="*60)

### Understanding the Results

#### Test 1 ($200) - ALLOWED ✅

**What happened:**
1. Agent sent request to gateway
2. Policy engine evaluated: `200 <= 1000` → **TRUE**
3. Request allowed through to Lambda
4. Lambda executed and processed refund
5. Success response returned to agent

**Key Point:** The Lambda function executed because the policy allowed it.

#### Test 2 ($2000) - DENIED ❌

**What happened:**
1. Agent sent request to gateway
2. Policy engine evaluated: `2000 <= 1000` → **FALSE**
3. Request **blocked** by policy engine
4. Lambda **never executed**
5. Policy denial response returned to agent

**Key Point:** The Lambda function never ran - the policy blocked it at the gateway level.

### Policy Enforcement in Action

This demonstrates the power of AgentCore Policy:
- ✅ **Declarative Security**: Policy defined separately from code
- ✅ **Runtime Enforcement**: Evaluated in real-time for every request
- ✅ **Zero Trust**: Even the AI agent can't bypass the policy
- ✅ **Audit Trail**: All decisions logged for compliance

## Clean Up
To clean up the resources of policy engines and policies, it is done in the following order:
1. Delete the association of the policy engine on the gateway by using the update_gateway CLI and passing in a empty policy engine

2. Delete all the policies in the policy engine

3. Delete the policy engine